In [14]:
import torch as t
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
device = 'cuda' if t.cuda.is_available() else 'cpu'

In [2]:
# Defining data transformations
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

In [3]:
# Loading the MNIST training dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)

In [4]:
#length of the training dataset
num_samples = len(train_dataset)
print("Total number of training samples:", num_samples)

Total number of training samples: 60000


In [5]:
# Creating DataLoader for the training dataset
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

In [6]:
train = enumerate(train_loader)
batch_idx, (input, label) = next(train)

In [7]:
input.shape

torch.Size([64, 1, 28, 28])

In [8]:
label.shape

torch.Size([64])

In [19]:
testset = datasets.MNIST('mnist_data/', download=True, train=False, transform=transform)

In [20]:
test_loader = DataLoader(testset, batch_size = batch_size, shuffle=False)

In [21]:
class Mnist(nn.Module):
    def __init__(self):
        super(Mnist,self).__init__()
        self.linear1 = nn.Linear(784, 64) 
        self.linear2 = nn.Linear(64, 32) 
        self.final = nn.Linear(32, 10)
        self.relu = nn.ReLU()

    def forward(self, img):
        x = img.view(-1, 784)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.final(x)
        return x
mnist = Mnist()

In [22]:
cross_el = nn.CrossEntropyLoss()
optimizer = t.optim.Adam(mnist.parameters(), lr=0.001)
epoch = 10

for epoch in range(epoch):
    for data in train_loader:
        x, y = data
        optimizer.zero_grad()
        output = mnist(x.view(-1, 28 * 28) )
        loss = cross_el(output, y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item() / len(train_loader)}")

Epoch 1, Loss: 0.00020782400105299472
Epoch 2, Loss: 9.822820041225409e-05
Epoch 3, Loss: 5.6247864323638396e-05
Epoch 4, Loss: 4.843429429953032e-05
Epoch 5, Loss: 2.9373504936313832e-05
Epoch 6, Loss: 1.714766017600163e-05
Epoch 7, Loss: 1.2806025959217727e-05
Epoch 8, Loss: 1.1264384269460177e-05
Epoch 9, Loss: 1.1545109516903281e-05
Epoch 10, Loss: 1.5633473637452257e-05


In [23]:
correct = 0
total = 0
with t.no_grad():
    for data in test_loader:
        x, y = data
        output = mnist(x.view(-1, 28 * 28))
        for idx, i in enumerate(output):
            if t.argmax(i) == y[idx]:
                correct +=1
            total +=1
print(f'accuracy: {round(correct/total, 3) * 100} ')

accuracy: 96.5 
